In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

2.18.0


In [2]:
# Wczytaj dane treningowe i testowe
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'

train_set = pd.read_csv(data_url, sep=", ",header = None)
test_set = pd.read_csv(test_url, sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

train = train_set.replace('?', np.nan).dropna()
test = test_set.replace('?', np.nan).dropna()

train_set.head()

<ipython-input-2-afb2aa67175a>:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_set = pd.read_csv(data_url, sep=", ",header = None)
<ipython-input-2-afb2aa67175a>:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_set = pd.read_csv(test_url, sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
dataset = pd.concat([train,test])

dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0,'<=50K':0, '>50K.':1, '>50K':1})

dataset.drop(["fnlwgt"],axis=1,inplace=True)

dataset.drop(["education"],axis=1,inplace=True)

x = dataset.groupby('native_country')["wage_class"].mean()

d = dict(pd.cut(x[x.index!=" United-States"],5,labels=range(5)))

dataset['native_country'] = dataset['native_country'].replace(d)

dataset = pd.get_dummies(dataset,drop_first=True)

train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class",axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class",axis=1)
y_test = test.wage_class

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

print(X_train.shape)
print(X_test.shape)

<ipython-input-3-3d44143aff24>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0,'<=50K':0, '>50K.':1, '>50K':1})
<ipython-input-3-3d44143aff24>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['native_country'] = dataset['native_country'].replace(d)


(30162, 41)
(15060, 41)


In [4]:
keras.backend.clear_session()

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=(41,),**kwargs):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [6]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier
keras_class = KerasClassifier(build_model)


In [11]:
from keras.callbacks import EarlyStopping
keras_class.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)])

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


849/849 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7457 - loss: 2.2377 - val_accuracy: 0.8011 - val_loss: 1.5354
Epoch 2/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7805 - loss: 2.0401 - val_accuracy: 0.7574 - val_loss: 2.4945
Epoch 3/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7733 - loss: 2.1477 - val_accuracy: 0.7958 - val_loss: 1.4004
Epoch 4/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8099 - loss: 1.0913 - val_accuracy: 0.7981 - val_loss: 1.1400
Epoch 5/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7842 - loss: 1.8713 - val_accuracy: 0.6904 - val_loss: 4.5123
Epoch 6/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7501 - loss: 3.4152 - val_accuracy: 0.7786 - val_loss: 3.1476
Epoch 7/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7961 - loss: 2.7993 - val_accuracy: 0.8064 - val_loss: 2.4343
Epoch 8/100
849/849 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7826 - loss: 2.9426 - val_accuracy: 0.7749

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

KerasClassifier(
	model=<function build_model at 0x7b7c49fa7600>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

# Zad

Wykonajmy RandomizedSearchCV

In [12]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100)
}


rnd_search_cv = RandomizedSearchCV(
    keras_class,
    param_distributions=param_distribs,
    n_iter=10,
    cv=3,
    verbose=1
)

rnd_search_cv.fit(X_train, y_train, validation_split=0.1, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
rnd_search_cv.best_params_

# Zad

Wykonaj RandomizedSearchCV na danych FashionMINT


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0


param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100)
}

def build(n_hidden=1, n_neurons=30, learning_rate=3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(28,28)))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(10,activation='softmax'))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

keras_class = KerasClassifier(build)

rnd_search_cv = RandomizedSearchCV(
    keras_class,
    param_distributions=param_distribs,
    n_iter=10,
    cv=3,
    verbose=1
)

rnd_search_cv.fit(X_train, y_train, validation_split=0.1, callbacks=[keras.callbacks.EarlyStopping(patience=5)])
